# Variant calling analysis Killswitch circuit 

Defining variants (SNPs, insertions and deletions) for a set of HiSeq samples exploring the variability in populations of *M. pneumoniae* transformed with a killswitch cassette (C5 samples) or two (C20 samples); sequencing performed at different selection passages (2, 3, 15; 2 replicates in each condition and cassette) and a pair of replicates at passage 3 were the cassette was induced by IPTG.

**Authors:** [Samuel Miravet-Verde](mailto:samuel.miravet@crg.eu) and [Alicia Broto](mailto:alicia.broto@crg.eu) 

**Last update:** 14/06/2021

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys, os
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scripts
from collections import Counter
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation

sns.set_style("whitegrid", {'axes.grid' : False})


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# General info about the genome references and the location of the cassettes in the genomes
seq_record5 = SeqIO.read('./data/C5_refSeq.gb', "genbank")
seq_record20 = SeqIO.read('./data/C20_refSeq.gb', "genbank")
genome_C5 = str(seq_record5.seq)
genome_C20 = str(seq_record5.seq)

cass_par = [565510, 572124] # Present in C5 and C20
cass_3b = [780850, 787445]  # Only in C20
cass_26 = [574473, 581077]
cass_262 = [79542, 88496]


# Annotation information to work faster with dataframes
ncbi = pd.read_csv('./data/mpn_annotation.csv', sep='\t', header=None) # Genome annotation
ncbi.columns = ['gene', 'start', 'end', 'strand']
ncbi.set_index('gene', inplace=True)
gold = pd.read_csv('./data/goldsets.csv', sep='\t')  # Gold set 

posN = []
posE = []
for gene, cat in zip(gold['gene'], gold['class']):
    if cat=='E':
        posE += range(ncbi.loc[gene][0], ncbi.loc[gene][1]+1)
    else:
        posN += range(ncbi.loc[gene][0], ncbi.loc[gene][1]+1)
        
# Define the different set of positions in the genome were we will perform the analysis
posN = set(posN)
posE = set(posE)
cas1 = set(range(cass_par[0], cass_par[1]+1))
cas2 = set(range(cass_3b[0], cass_3b[1]+1))
cas12 = cas1.union(cas2)
cas31 = set(range(cass_26[0], cass_26[1]+1))
cas32 = set(range(cass_262[0], cass_262[1]+1))
cas33 = cas31.union(cas32)

geno = set(range(1, 816395))

ingene = []
for st, en in zip(ncbi['start'], ncbi['end']):
    ingene+=list(range(st, en))
ingene = set(ingene)
outgene = geno.difference(ingene)

## 1. Map variations

We will be using [Snippy](https://github.com/tseemann/snippy) for the first steps related to sequence mapping. To run it in parallel for all the samples we create a input file:

In [147]:
### 1. Make a multifile to map with snippy-multi.
files = glob.glob('../data/C[5|20|26]*/*_R1_*') # The gre expression refers to the raw sequencing files located in data (this has to point to the directory where fastq files are stored)

fo1 = open('./tmp_files/snippy_input1.txt', 'w')
fo2 = open('./tmp_files/snippy_input2.txt', 'w')
fo3 = open('./tmp_files/snippy_input3.txt', 'w')
for f in files:
    ide = f.split('/')[2:]
    if 'C5' in f:
        ide = ide[0].replace('C5', '')+'_'+'_'.join(ide[1].split('_')[:-3])
        fo1.write('{}\t{}\t{}\n'.format(ide, f, f.replace('_R1_', '_R2_')))
    elif 'C20' in f:
        ide = ide[0].replace('C20', '')+'_'+'_'.join(ide[1].split('_')[:-3])
        fo2.write('{}\t{}\t{}\n'.format(ide, f, f.replace('_R1_', '_R2_')))
    else:
        ide = ide[0].replace('C26', '')+'_'+'_'.join(ide[1].split('_')[:-3])
        fo3.write('{}\t{}\t{}\n'.format(ide, f, f.replace('_R1_', '_R2_')))        
fo1.close()
fo2.close()
fo3.close()
files

['../data/C20p3/B_S1_L001_R1_001.fastq.gz',
 '../data/C20p3/B_S2_L001_R1_001.fastq.gz',
 '../data/C5p3/D_S2_L001_R1_001.fastq.gz',
 '../data/C5p3/D_S3_L001_R1_001.fastq.gz',
 '../data/C5p3IPTG/A_S1_L001_R1_001.fastq.gz',
 '../data/C5p3IPTG/A_S1b_L001_R1_001.fastq.gz',
 '../data/C20p15/7_S3_L001_R1_001.fastq.gz',
 '../data/C20p15/7_S5_L001_R1_001.fastq.gz',
 '../data/C5p15/F_S5_L001_R1_001.fastq.gz',
 '../data/C5p15/F_S1_L001_R1_001.fastq.gz',
 '../data/C5p2/2_S2_L001_R1_001.fastq.gz',
 '../data/C5p2/2_S3_L001_R1_001.fastq.gz',
 '../data/C5p2/2_S1_L001_R1_001.fastq.gz',
 '../data/C26p2/8_S2_L001_R1_001.fastq.gz',
 '../data/C26p2/8_S3_L001_R1_001.fastq.gz',
 '../data/C20p2/6_S2_L001_R1_001.fastq.gz',
 '../data/C20p2/6_S4_L001_R1_001.fastq.gz',
 '../data/C26p15/9_S7_L001_R1_001.fastq.gz',
 '../data/C26p15/9_S3_L001_R1_001.fastq.gz']

Now run the command in the snippy environment:

```bash
conda activate snippy
snippy-multi ./tmp_files/snippy_input.txt --ref ./data/C5_refSeq.gb --cpus 4 --report >  ./snippy_run.sh
sh ./snippy_run.sh
```

In [5]:
# Listed samples in order
order1 = ['p2_2_S1','p2_2_S2','p2_2_S3','p3_D_S2', 'p3_D_S3','p15_F_S1','p15_F_S5', 'p3IPTG_A_S1', 'p3IPTG_A_S1b']
order2 = ['p2_6_S2','p2_6_S4','p3_B_S1','p3_B_S2','p15_7_S3','p15_7_S5']
order3 = ['p2_8_S2','p2_8_S3','p15_9_S3','p15_9_S7']

In [12]:
# freebayes-parallel reference/ref.txt 4 -p 2 -P 0 -C 2 -F 0.05 --min-coverage 10 --min-repeat-entropy 1.0 -q 13 -m 60 --strict-vcf   -f reference/ref.fa snps.bam > snps.raw.vcf
for fil in glob.glob('../p*/*.bam'):
    # again this has to point to directories were snipy output is stored
    ide = fil.split('/')[1]
    #print('freebayes-parallel {}/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.01 -q 13 -m 60 --strict-vcf -f {}/reference/ref.fa {} > filter10/{}.raw.vcf'.format(ide, ide, fil, ide))
    #print('freebayes-parallel {}/reference/ref.txt 4 -p 1 -P 0 -C 2 -F 0.05 -q 13 -m 60 --strict-vcf -f {}/reference/ref.fa {} > filter50/{}.raw.vcf'.format(ide, ide, fil, ide))
    if ide in order1:
        pass
        print('freebayes-parallel {}/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f {}/reference/ref.fa {} > filter1/{}.raw.vcf'.format(ide, ide, fil, ide))
    elif ide in order2:
        pass
        print('freebayes-parallel {}/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f {}/reference/ref.fa {} > filter2/{}.raw.vcf'.format(ide, ide, fil, ide))
    else:
        print('freebayes-parallel {}/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f {}/reference/ref.fa {} > filter3/{}.raw.vcf'.format(ide, ide, fil, ide))
        


freebayes-parallel p2_8_S2/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f p2_8_S2/reference/ref.fa ../p2_8_S2/snps.bam > filter3/p2_8_S2.raw.vcf
freebayes-parallel p2_8_S3/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f p2_8_S3/reference/ref.fa ../p2_8_S3/snps.bam > filter3/p2_8_S3.raw.vcf
freebayes-parallel p15_9_S3/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f p15_9_S3/reference/ref.fa ../p15_9_S3/snps.bam > filter3/p15_9_S3.raw.vcf
freebayes-parallel p15_9_S7/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f p15_9_S7/reference/ref.fa ../p15_9_S7/snps.bam > filter3/p15_9_S7.raw.vcf


To get the total number of mapped reads:

```python
for fil in glob.glob('./p*/*.bam'):
    ide = fil.split('/')[1]
    # get the total number of reads of a BAM file (may include unmapped and duplicated multi-aligned reads)
    print('samtools view -c {} >> total_readcount.txt'.format(fil))
    print('samtools view -c {} -F 260 >> total_readcount_mapped.txt'.format(fil))
```

Finally we annotate the effect of the variants using SnpEff. This requires to add the custom annotation to the sources, specifying that the translation code is set to 4 for *M. pneumoniae*.

1. Download SnpEff and edited the config (in software_crg/SnpEff) to include the C5, C20, C26 refseq references
2. Build the custom db:
```bash
java -jar snpEff.jar build -genbank -v C5ali
java -jar snpEff.jar build -genbank -v C20ali
java -jar snpEff.jar build -genbank -v C26ali
```
3. Annotate vcf files in bash (in directory of the software)
```bash
for fil in `ls ./filter1/p*.vcf`; do java -Xmx8g -jar snpEff.jar C5ali $fil > $fil.ann; done
for fil in `ls ./filter2/p*.vcf`; do java -Xmx8g -jar snpEff.jar C20ali $fil > $fil.ann; done
for fil in `ls ./filter3/p*.vcf`; do java -Xmx8g -jar snpEff.jar C26ali $fil > $fil.ann; done
```


## 2. Data loading 

We will parse the variations to keep the columns of interest to perform the exploratory analysis. Dataframes include the following information (by column header):
- SAMPLE: sample identifier includes information about the passage (e.g. p3IPTG is the passage 3 induced)
- PASS: passage (2,3 or 15)
- POS: genome loci (base pair position) where the variant is found
- QUAL: estimate of the probability that there is a polymorphism at the loci described by the record
- TOT: total number of reads covering a loci
- REFN: total number of reads covering a loci matching the reference
- ALTN: number of reads presenting a variant
- REF: reference sequence
- ALT: alternative sequence
- EFF: potential effect of the variant as given by snpeff
- IMPACT: potential impact of the variant, can be LOW (synonimous mutations), MODERATE (missense), HIGH (non-synonimous mutations, start lost, stop lost), or MODIFIER (when it occurs in an intergenic region)
- AFF: gene affected
- MUT: mutation (in nucleotide if not coding in amino acid if coding)
- ANN_TYPE: annotation of the loci, can be intergenic, gene, or cassette (covering the principal elements in the cassette)


In [6]:
# This will concat all the annotated vcf files keeping the fields of interest
data1 = scripts.parse_variations('./filter1/*.raw.vcf.ann', order1, genome=5, fname='./results/data1.pickle')
data2 = scripts.parse_variations('./filter2/*.raw.vcf.ann', order2, genome=20, fname='./results/data2.pickle')
data3 = scripts.parse_variations('./filter3/*.raw.vcf.ann', order3, genome=26, fname='./results/data3.pickle')

Processing data for genome 5
Cleaning data for genome 5
Memory usage of properties dataframe is : 62.0408935546875  MB
******************************
Column:  PASS
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  POS
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  QUAL
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  TOT
dtype before:  int64
dtype after:  uint16
******************************
******************************
Column:  REFN
dtype before:  int64
dtype after:  uint16
******************************
******************************
Column:  ALTN
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  FRAC
dtype before:  float64
dtype after:  float32
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:

In [151]:
len(cas33)

15560

We can now explore the variants, for example to extract those present in the cassette with high impact (non-synonimous) in samples induced (assigned as passage 18 despite they are passage 3), could be retrieved as:

In [34]:
# Example:
## Notice the information is grouped by position, we will deal with this in section 3. 
data1[(data1['ANN_TYPE']=='cassette') & (data1['IMPACT']=='HIGH') & (data1['PASS']==18)].sort_values('FRAC')

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
443045,p3IPTG_A_S1,18,566372,2.184800e-15,677,676,1,0.147929,CTA,TTT,stop_gained,HIGH,LacI4,p.LeuGly102*,cassette
443082,p3IPTG_A_S1,18,566622,0.000000e+00,647,646,1,0.154799,A,C,stop_gained,HIGH,LacI4,p.Tyr19*,cassette
443002,p3IPTG_A_S1,18,565989,2.863080e-15,582,581,1,0.172117,TCCCTCG,TCCTCG,frameshift_variant,HIGH,LacI4,p.Gly230fs,cassette
443294,p3IPTG_A_S1,18,568518,7.700020e-15,577,576,1,0.173611,G,A,stop_gained,HIGH,cas9B,p.Gln1101*,cassette
443402,p3IPTG_A_S1,18,569451,6.132760e-15,572,571,1,0.175131,C,A,stop_gained,HIGH,cas9B,p.Glu790*,cassette
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443663,p3IPTG_A_S1,18,571681,0.000000e+00,481,469,12,2.558635,ATTTTTTTTGATA,ATTTTTTTGATA,frameshift_variant,HIGH,cas9B,p.Asn46fs,cassette
495218,p3IPTG_A_S1b,18,571681,7.895900e-15,191,182,9,4.945055,ATTTTTTTTGATACT,ATTTCTTTAGAAACA,stop_gained,HIGH,cas9B,p.SerIleLysLys42*,cassette
495059,p3IPTG_A_S1b,18,569127,8.076710e-15,235,210,25,11.904762,CAAACT,CAACT,frameshift_variant,HIGH,cas9B,p.Phe897fs,cassette
443366,p3IPTG_A_S1,18,569125,0.000000e+00,534,463,71,15.334773,ATCAAACT,GTCAACC,frameshift_variant&missense_variant,HIGH,cas9B,p.Lys896fs,cassette


In [35]:
# For C20 we have snpcalls2 in the same format
data2[(data2['ANN_TYPE']=='cassette') & (data2['IMPACT']=='HIGH') & (data2['PASS']==3)].sort_values('FRAC')

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
225393,p3_B_S1,3,786189,0.000000e+00,280,279,1,0.358423,TTAC,CTAT,stop_gained,HIGH,cas2 cat,p.Gln209*,cassette
225340,p3_B_S1,3,785573,2.991160e-15,253,252,1,0.396825,GAAAAAAATCAC,GAAAAAAAATCAC,frameshift_variant,HIGH,cas2 cat,p.Ile5fs,cassette
225346,p3_B_S1,3,785643,0.000000e+00,252,251,1,0.398406,C,T,stop_gained,HIGH,cas2 cat,p.Gln26*,cassette
264083,p3_B_S2,3,566670,0.000000e+00,226,225,1,0.444444,CTTTGC,CTTGC,frameshift_variant,HIGH,cas1 LacI4,p.Lys3fs,cassette
225402,p3_B_S1,3,787394,0.000000e+00,150,149,1,0.671141,AGCG,TGCA,stop_lost,HIGH,cas2 lacI4,p.TerArg373LeuGlnext*?,cassette
277049,p3_B_S2,3,785709,1.702880e-15,258,256,2,0.781250,GTA,GA,frameshift_variant,HIGH,cas2 cat,p.Val48fs,cassette
277050,p3_B_S2,3,785714,1.642610e-15,255,253,2,0.790514,GAAAAATAAGC,GAAAATAAGC,frameshift_variant,HIGH,cas2 cat,p.Asn51fs,cassette
209632,p3_B_S1,3,566676,1.994980e-14,226,224,2,0.892857,CATA,TATT,start_lost,HIGH,cas1 LacI4,p.Met1?,cassette
209630,p3_B_S1,3,566634,2.558530e-14,112,111,1,0.900901,AGGCT,AGGGCT,frameshift_variant,HIGH,cas1 LacI4,p.Val16fs,cassette
225397,p3_B_S1,3,786318,0.000000e+00,98,97,1,1.030928,GCCTG,GCCCTG,frameshift_variant,HIGH,cas2 lacI4,p.Val16fs,cassette


In [36]:
# For C20 we have snpcalls2 in the same format
data3[(data3['ANN_TYPE']=='cassette') & (data3['IMPACT']=='HIGH') & (data3['PASS']==2)].sort_values('FRAC')

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
124456,p2_8_S3,2,575262,4.520520e-15,613,612,1,0.163399,AGTA,TGTT,stop_gained,HIGH,LacI4,p.LeuLeu126*,cassette
124447,p2_8_S3,2,575215,3.737000e-15,612,611,1,0.163666,G,A,stop_gained,HIGH,LacI4,p.Gln143*,cassette
124410,p2_8_S3,2,574999,0.000000e+00,587,586,1,0.170648,T,A,stop_gained,HIGH,LacI4,p.Lys215*,cassette
124493,p2_8_S3,2,575509,7.700020e-15,577,576,1,0.173611,T,A,stop_gained,HIGH,LacI4,p.Lys45*,cassette
124396,p2_8_S3,2,574915,0.000000e+00,534,533,1,0.187617,G,A,stop_gained,HIGH,LacI4,p.Gln243*,cassette
124398,p2_8_S3,2,574924,1.536900e-15,530,529,1,0.189036,G,A,stop_gained,HIGH,LacI4,p.Gln240*,cassette
46650,p2_8_S2,2,575585,0.000000e+00,524,523,1,0.191205,A,T,stop_gained,HIGH,LacI4,p.Tyr19*,cassette
46569,p2_8_S2,2,574905,0.000000e+00,403,402,1,0.248756,TTTA,CTTT,stop_gained,HIGH,LacI4,p.LeuAsn245*,cassette
124342,p2_8_S3,2,574518,0.000000e+00,347,346,1,0.289017,T,C,stop_lost,HIGH,LacI4,p.Ter375Trpext*?,cassette
124343,p2_8_S3,2,574519,1.536900e-15,322,321,1,0.311526,A,C,stop_lost,HIGH,LacI4,p.Ter375Gluext*?,cassette


In [81]:
# Save supplementary 1
sup1 = './results/suptableS1.xlsx'
if not os.path.isfile(sup1):
    writer = pd.ExcelWriter(sup1, engine='xlsxwriter')
    for c, data in zip(['C5_', 'C20_', 'C_26'], [data1, data2, data3]):
        for sample in set(data['SAMPLE']):
            subdata = data[data['SAMPLE']==sample].copy()
            subdata.to_excel(writer, sheet_name=c+sample)
    writer.save()

## 3. Rate and fraction of mutations in cassette compared to other distributions (supplementary figure 1)

Explore the frequency at which we found a variant within the cassette compared to other types of annotations and also their representation within the population by means of the fraction values. 

In [7]:
def plot_percentage(df, genome=5):
    """ Plot to show the percentage of variants mapping to each type of annotation """
    rs = {}
    c = 0
    lens = {}
    lens['gene'] = len(ingene)
    lens['intergenic'] = len(outgene)
    lens['essential'] = len(posE)
    lens['non-essential'] = len(posN)
    if genome==5:
        lens['chromosome'] = len(genome_C5)
        lens['cassette'] = len(cas1)
    elif genome==20:
        lens['chromosome'] = len(genome_C20)
        lens['cassette'] = len(cas12)
    else:
        lens['chromosome'] = len(genome_C26)
        lens['cassette'] = len(cas33)
    # Extract positions in passage 2
    for pas in set(df['PASS']):
        for sample in set(df[df['PASS']==pas]['SAMPLE']):
            for impact in set(df['IMPACT']):
                rs[c] = [pas, 
                         100*len(set(df[(df['PASS']==pas) & (df['IMPACT']==impact)  & (df['SAMPLE']==sample)]['POS']))/lens['chromosome'],
                         sample, 'TOTAL', impact]
                c+=1
            rs[c] = [pas, 100*len(set(df[(df['PASS']==pas)  & (df['SAMPLE']==sample)]['POS']))/lens['chromosome'],sample, 'chromosome', 'TOTAL']
            c+=1
            for ann in set(df['ANN_TYPE']):
                if ann in lens:
                    rs[c] = [pas, 
                             100*len(set(df[(df['PASS']==pas) & (df['ANN_TYPE']==ann)  & (df['SAMPLE']==sample)]['POS']))/lens[ann],
                             sample, ann, 'TOTAL']
                    c+=1
                    for impact in set(df['IMPACT']):
                        rs[c] = [pas, 
                                 100*len(set(df[(df['PASS']==pas) & (df['ANN_TYPE']==ann) & (df['IMPACT']==impact)  & (df['SAMPLE']==sample)]['POS']))/lens[ann],
                                 sample, ann, impact]
                        c+=1
    rs = pd.DataFrame.from_dict(rs, orient='index')
    rs.columns = ['PASS', 'PERC', 'sample', 'annotation', 'IMPACT']
    rs['HUE'] = rs['annotation']+rs['IMPACT']
    return rs.sort_values('annotation')

subdf1 = data1[(data1['FRAC']<100) & (data1['ALTN']>=2)].sort_values(['ANN_TYPE', 'PASS']).copy()
subdf2 = data2[(data2['FRAC']<100) & (data2['ALTN']>=2)].sort_values(['ANN_TYPE', 'PASS']).copy()
subdf3 = data3[(data3['FRAC']<100) & (data3['ALTN']>=2)].sort_values(['ANN_TYPE', 'PASS']).copy()

plot1 = plot_percentage(subdf1)
plot2 = plot_percentage(subdf2)
plot3 = plot_percentage(subdf3)

subdf1['log2Frac'] = np.log2(subdf1['FRAC'])
subdf1['Condition'] = ['{}'.format(i) if i!=18 else '3IPTG' for i in subdf1['PASS']]
subdf1['annotation'] = subdf1['ANN_TYPE']

subdf2['log2Frac'] = np.log2(subdf2['FRAC'])
subdf2['Condition'] = ['{}'.format(i) if i!=18 else '3IPTG' for i in subdf2['PASS']]
subdf2['annotation'] = subdf2['ANN_TYPE']

subdf3['log2Frac'] = np.log2(subdf3['FRAC'])
subdf3['Condition'] = ['{}'.format(i) if i!=18 else '3IPTG' for i in subdf3['PASS']]
subdf3['annotation'] = subdf3['ANN_TYPE']

plt.close('all')
plt.figure(figsize=(10, 10))
#plt.subplot(3,3,1)
#sns.countplot(x='Condition', hue='annotation', data=subdf1[(subdf1['ANN_TYPE'].isin(['cassette', 'gene', 'intergenic']))])

plt.subplot(3,2,1)
sns.barplot(x='PASS', y='PERC', hue='annotation', data=plot1[(plot1['IMPACT']=='TOTAL') & (plot1['annotation'].isin(['cassette', 'gene', 'intergenic']))])
plt.xticks([0,1,2,3], ['2', '3', '15', '3IPTG'])
plt.xlabel('Condition')
plt.ylabel('Percentage')
plt.title('Variant rate per base pair C5 samples')

plt.subplot(3,2,2)
sns.boxplot(y='log2Frac', x='Condition', hue='annotation', data=subdf1[(subdf1['log2Frac']>0) & (subdf1['ANN_TYPE'].isin(['cassette', 'gene', 'intergenic']))])
plt.xlabel('Condition')
plt.ylabel('log2(Fraction)')
plt.title('Alternative Fraction C5 samples')

##plt.ylim(0,3.2)
#plt.subplot(3,3,4)
#sns.countplot(x='PASS', hue='annotation', data=subdf2[(subdf2['ANN_TYPE'].isin(['cassette', 'gene', 'intergenic']))])
#plt.xlabel('Condition')

plt.subplot(3,2,3)
sns.barplot(x='PASS', y='PERC', hue='annotation', data=plot2[(plot2['IMPACT']=='TOTAL') & (plot2['annotation'].isin(['cassette', 'gene', 'intergenic']))])
plt.xlabel('Condition')
plt.ylabel('Percentage')
plt.title('Variant rate per base pair C20 samples')
#plt.ylim(0,3.2)

plt.subplot(3,2,4)
sns.boxplot(y='log2Frac', x='PASS', hue='annotation', data=subdf2[(subdf2['log2Frac']>0) & (subdf2['ANN_TYPE'].isin(['cassette', 'gene', 'intergenic']))])
plt.xlabel('Condition')
plt.ylabel('log2(Fraction)')
plt.title('Alternative Fraction  C20 samples')

#plt.subplot(3,3,7)
#sns.countplot(x='PASS', hue='annotation', data=subdf3[(subdf3['ANN_TYPE'].isin(['cassette', 'gene', 'intergenic']))])
#plt.xlabel('Condition')

plt.subplot(3,2,5)
sns.barplot(x='PASS', y='PERC', hue='annotation', data=plot3[(plot3['IMPACT']=='TOTAL') & (plot3['annotation'].isin(['cassette', 'gene', 'intergenic']))])
plt.xlabel('Condition')
plt.ylabel('Percentage')
plt.title('Variant rate per base pair C26 samples')
#plt.ylim(0,3.2)

plt.subplot(3,2,6)
sns.boxplot(y='log2Frac', x='PASS', hue='annotation', data=subdf3[(subdf3['log2Frac']>0) & (subdf3['ANN_TYPE'].isin(['cassette', 'gene', 'intergenic']))])
plt.xlabel('Condition')
plt.ylabel('log2(Fraction)')
plt.title('Alternative Fraction  C26 samples')

plt.tight_layout()
plt.savefig('./results/supfigS1.svg')
plt.savefig('./results/supfigS1.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Let's calculate the statistics for this comparisons:

In [153]:
from scipy.stats import mannwhitneyu
percentage_pvalues = {}
fraction_pvalues = {}

perc1 = plot1[(plot1['IMPACT']=='TOTAL') & (plot1['annotation'].isin(['cassette', 'gene', 'intergenic']))].copy()
perc2 = plot2[(plot2['IMPACT']=='TOTAL') & (plot2['annotation'].isin(['cassette', 'gene', 'intergenic']))].copy()
perc3 = plot3[(plot3['IMPACT']=='TOTAL') & (plot3['annotation'].isin(['cassette', 'gene', 'intergenic']))].copy()

frac1 = subdf1[(subdf1['log2Frac']>0) & (subdf1['ANN_TYPE'].isin(['cassette', 'gene', 'intergenic']))].copy()
frac2 = subdf2[(subdf2['log2Frac']>0) & (subdf2['ANN_TYPE'].isin(['cassette', 'gene', 'intergenic']))].copy()
frac3 = subdf3[(subdf3['log2Frac']>0) & (subdf3['ANN_TYPE'].isin(['cassette', 'gene', 'intergenic']))].copy()

c = 0
for pas in set(perc1['PASS']):
    x = perc1[(perc1['PASS']==pas) & (perc1['annotation']=='cassette')]['PERC']
    for annot in ['gene', 'intergenic']:
        y = perc1[(perc1['PASS']==pas) & (perc1['annotation']==annot)]['PERC']
        percentage_pvalues[c] = ['C5', pas, annot, round(mannwhitneyu(x, y)[1]/2, 4)]
        c+=1
for pas in set(perc2['PASS']):
    x = perc2[(perc2['PASS']==pas) & (perc2['annotation']=='cassette')]['PERC']
    for annot in ['gene', 'intergenic']:
        y = perc2[(perc2['PASS']==pas) & (perc2['annotation']==annot)]['PERC']
        percentage_pvalues[c] = ['C20', pas, annot,round(mannwhitneyu(x, y)[1]/2, 4)]
        c+=1
for pas in set(perc3['PASS']):
    x = perc3[(perc3['PASS']==pas) & (perc3['annotation']=='cassette')]['PERC']
    for annot in ['gene', 'intergenic']:
        y = perc3[(perc3['PASS']==pas) & (perc3['annotation']==annot)]['PERC']
        percentage_pvalues[c] = ['C26', pas, annot,round(mannwhitneyu(x, y)[1]/2, 4)]
        c+=1
        
c = 0
for pas in set(frac1['PASS']):
    x = frac1[(frac1['PASS']==pas) & (frac1['annotation']=='cassette')]['log2Frac']
    for annot in ['gene', 'intergenic']:
        y = frac1[(frac1['PASS']==pas) & (frac1['annotation']==annot)]['log2Frac']
        fraction_pvalues[c] = ['C5', pas, annot, round(mannwhitneyu(x, y)[1]/2, 4)]
        c+=1
for pas in set(frac2['PASS']):
    x = frac2[(frac2['PASS']==pas) & (frac2['annotation']=='cassette')]['log2Frac']
    for annot in ['gene', 'intergenic']:
        y = frac2[(frac2['PASS']==pas) & (frac2['annotation']==annot)]['log2Frac']
        fraction_pvalues[c] = ['C20', pas, annot,round(mannwhitneyu(x, y)[1]/2, 4)]
        c+=1
for pas in set(frac3['PASS']):
    x = frac3[(frac3['PASS']==pas) & (frac3['annotation']=='cassette')]['log2Frac']
    for annot in ['gene', 'intergenic']:
        y = frac3[(frac3['PASS']==pas) & (frac3['annotation']==annot)]['log2Frac']
        fraction_pvalues[c] = ['C26', pas, annot,round(mannwhitneyu(x, y)[1]/2, 4)]
        c+=1

In [154]:
print('Average rate of mutations per base in the cassette', np.mean(perc1[(perc1['annotation']=='cassette')]['PERC']))
print('Average rate of mutations per base in the intergenic', np.mean(perc1[(perc1['annotation']=='intergenic')]['PERC']))
print('Average rate of mutations per base in the gene', np.mean(perc1[(perc1['annotation']=='gene')]['PERC']))

print('Average rate of mutations per base in the cassette', np.mean(perc2[(perc2['annotation']=='cassette')]['PERC']))
print('Average rate of mutations per base in the intergenic', np.mean(perc2[(perc2['annotation']=='intergenic')]['PERC']))
print('Average rate of mutations per base in the gene', np.mean(perc2[(perc2['annotation']=='gene')]['PERC']))

print('Average rate of mutations per base in the cassette', np.mean(perc3[(perc3['annotation']=='cassette')]['PERC']))
print('Average rate of mutations per base in the intergenic', np.mean(perc3[(perc3['annotation']=='intergenic')]['PERC']))
print('Average rate of mutations per base in the gene', np.mean(perc3[(perc3['annotation']=='gene')]['PERC']))

Average rate of mutations per base in the cassette 2.825228856974889
Average rate of mutations per base in the intergenic 1.8428205685101453
Average rate of mutations per base in the gene 1.4096396774343913
Average rate of mutations per base in the cassette 0.9750566893424035
Average rate of mutations per base in the intergenic 2.42446081044616
Average rate of mutations per base in the gene 1.8454062368899455
Average rate of mutations per base in the cassette 2.671957671957672
Average rate of mutations per base in the intergenic 2.705737753470097
Average rate of mutations per base in the gene 2.0816319395888687


In [155]:
percentage_pvalues

{0: ['C5', 18, 'gene', 0.0613],
 1: ['C5', 18, 'intergenic', 0.1746],
 2: ['C5', 2, 'gene', 0.0476],
 3: ['C5', 2, 'intergenic', 0.0476],
 4: ['C5', 3, 'gene', 0.1746],
 5: ['C5', 3, 'intergenic', 0.1746],
 6: ['C5', 15, 'gene', 0.0613],
 7: ['C5', 15, 'intergenic', 0.0613],
 8: ['C20', 2, 'gene', 0.0613],
 9: ['C20', 2, 'intergenic', 0.0613],
 10: ['C20', 3, 'gene', 0.0613],
 11: ['C20', 3, 'intergenic', 0.0613],
 12: ['C20', 15, 'gene', 0.0613],
 13: ['C20', 15, 'intergenic', 0.0613],
 14: ['C26', 2, 'gene', 0.0613],
 15: ['C26', 2, 'intergenic', 0.1746],
 16: ['C26', 15, 'gene', 0.1746],
 17: ['C26', 15, 'intergenic', 0.1746]}

In [156]:
print('Average rate of mutations per base in the cassette', np.percentile(frac1[(frac1['annotation']=='cassette')  & (frac1['log2Frac']>0)]['FRAC'], 50))
print('Average rate of mutations per base in the intergenic', np.percentile(frac1[(frac1['annotation']=='intergenic') & (frac1['log2Frac']>0) ]['FRAC'], 50))
print('Average rate of mutations per base in the gene', np.percentile(frac1[(frac1['annotation']=='gene')& (frac1['log2Frac']>0)]['FRAC'], 50))

print('Average rate of mutations per base in the cassette', np.median(frac2[(frac2['annotation']=='cassette')]['FRAC']))
print('Average rate of mutations per base in the intergenic', np.median(frac2[(frac2['annotation']=='intergenic')]['FRAC']))
print('Average rate of mutations per base in the gene', np.median(frac2[(frac2['annotation']=='gene')]['log2Frac']))

print('Average rate of mutations per base in the cassette', np.median(frac3[(frac3['annotation']=='cassette')]['FRAC']))
print('Average rate of mutations per base in the intergenic', np.median(frac3[(frac3['annotation']=='intergenic')]['FRAC']))
print('Average rate of mutations per base in the gene', np.median(frac3[(frac3['annotation']=='gene')]['log2Frac']))

Average rate of mutations per base in the cassette 1.492537260055542
Average rate of mutations per base in the intergenic 1.5564202070236206
Average rate of mutations per base in the gene 1.470588207244873
Average rate of mutations per base in the cassette 1.5151515
Average rate of mutations per base in the intergenic 1.3513514
Average rate of mutations per base in the gene 0.38646838
Average rate of mutations per base in the cassette 1.3468167
Average rate of mutations per base in the intergenic 1.3513514
Average rate of mutations per base in the gene 0.36773175


In [157]:
fraction_pvalues

{0: ['C5', 18, 'gene', 0.0117],
 1: ['C5', 18, 'intergenic', 0.2241],
 2: ['C5', 2, 'gene', 0.0],
 3: ['C5', 2, 'intergenic', 0.0],
 4: ['C5', 3, 'gene', 0.1343],
 5: ['C5', 3, 'intergenic', 0.0988],
 6: ['C5', 15, 'gene', 0.1662],
 7: ['C5', 15, 'intergenic', 0.0021],
 8: ['C20', 2, 'gene', 0.0002],
 9: ['C20', 2, 'intergenic', 0.0016],
 10: ['C20', 3, 'gene', 0.0],
 11: ['C20', 3, 'intergenic', 0.0],
 12: ['C20', 15, 'gene', 0.0],
 13: ['C20', 15, 'intergenic', 0.002],
 14: ['C26', 2, 'gene', 0.2051],
 15: ['C26', 2, 'intergenic', 0.1272],
 16: ['C26', 15, 'gene', 0.0977],
 17: ['C26', 15, 'intergenic', 0.03]}

## 4. Location of the mutations within the cassettes (supplementary figure 2)

In [10]:
subdf1 = data1[(data1['FRAC']<100) & (data1['ALTN']>=2) & (data1['QUAL']>0.0)  & (data1['IMPACT']!='MODIFIER') & (~data1['AFF'].str.contains('pS'))].sort_values(['PASS', 'AFF']).copy()
subdf2 = data2[(data2['FRAC']<100) & (data2['ALTN']>=2) & (data2['QUAL']>0.0) & (data2['IMPACT']!='MODIFIER')  & (~data2['AFF'].str.contains('pS'))].sort_values(['PASS', 'AFF']).copy() 
subdf3 = data3[(data3['FRAC']<100) & (data3['ALTN']>=2) & (data3['QUAL']>0.0) & (data3['IMPACT']!='MODIFIER')  & (~data3['AFF'].str.contains('pS'))].sort_values(['PASS', 'AFF']).copy() 

sorter = ['HIGH', 'MODERATE', 'LOW']
subdf1.IMPACT = subdf1.IMPACT.astype("category")
subdf1.IMPACT.cat.set_categories(sorter, inplace=True)
subdf2.IMPACT = subdf2.IMPACT.astype("category")
subdf2.IMPACT.cat.set_categories(sorter, inplace=True)
subdf3.IMPACT = subdf3.IMPACT.astype("category")
subdf3.IMPACT.cat.set_categories(sorter, inplace=True)

plot1 = plot_percentage(subdf1)
plot2 = plot_percentage(subdf2)
plot3 = plot_percentage(subdf3)

subdf1['log2Frac'] = np.log2(subdf1['FRAC'])
subdf1['Condition'] = ['{}'.format(i) if i!=18 else '3IPTG' for i in subdf1['PASS']]
subdf1['annotation'] = subdf1['ANN_TYPE']

subdf2['log2Frac'] = np.log2(subdf2['FRAC'])
subdf2['Condition'] = ['{}'.format(i) if i!=18 else '3IPTG' for i in subdf2['PASS']]
subdf2['annotation'] = subdf2['ANN_TYPE']

subdf3['log2Frac'] = np.log2(subdf3['FRAC'])
subdf3['Condition'] = ['{}'.format(i) if i!=18 else '3IPTG' for i in subdf3['PASS']]
subdf3['annotation'] = subdf3['ANN_TYPE']

plt.close('all')
plt.figure(figsize=(10, 10))

# C5
plt.subplot(3,3,1)
sns.countplot(x='AFF', hue='PASS', data=subdf1[(subdf1['ANN_TYPE'].isin(['cassette']))])
plt.xticks(rotation=45, ha='right')
plt.legend(title="Passage")
plt.xlabel('Element')

plt.subplot(3,3,2)
sns.swarmplot(y='FRAC', x='AFF', hue='PASS', data=subdf1[(subdf1['log2Frac']>0) & (subdf1['ANN_TYPE'].isin(['cassette']))])
plt.xlabel('Element')
plt.ylabel('Fraction')
plt.legend(title="Passage")
plt.xticks(rotation=45, ha='right')
plt.title('C5 samples')
#plt.legend(bbox_to_anchor=(1.05, 1))

plt.subplot(3,3,3)
sns.swarmplot(x='AFF', y='FRAC', hue='IMPACT',data=subdf1[(subdf1['FRAC']>=1.5) & (subdf1['ANN_TYPE']=='cassette')  & (subdf1['IMPACT']!='MODIFIER')])
plt.xticks(rotation=45, ha='right')
plt.legend(title="Passage")
plt.xlabel('Element')
plt.ylabel('Fraction')
plt.legend(title="Impact")

#plt.ylim(0,3.2)
plt.subplot(3,3,4)
sns.countplot(x='AFF', hue='PASS', data=subdf2[(subdf2['ANN_TYPE'].isin(['cassette']))])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Element')
plt.legend(title="Passage")

plt.subplot(3,3,5)
sns.swarmplot(y='FRAC', x='AFF', hue='PASS', data=subdf2[(subdf2['log2Frac']>0) & (subdf2['ANN_TYPE'].isin(['cassette']))])
plt.xlabel('Element')
plt.ylabel('Fraction')
plt.xticks(rotation=45, ha='right')
plt.legend(title="Passage")
plt.title('C20 samples')
#plt.legend(bbox_to_anchor=(1.05, 1))

plt.subplot(3,3,6)
sns.swarmplot(x='AFF', y='FRAC', hue='IMPACT',data=subdf2[(subdf2['FRAC']>=1.5) & (subdf2['ANN_TYPE']=='cassette')  & (subdf2['IMPACT']!='MODIFIER')])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Element')
plt.ylabel('Fraction')
plt.legend(title="Impact")

# C26

plt.subplot(3,3,7)
sns.countplot(x='AFF', hue='PASS', data=subdf3[(subdf3['ANN_TYPE'].isin(['cassette']))])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Element')
plt.legend(title="Passage")

plt.subplot(3,3,8)
sns.swarmplot(y='FRAC', x='AFF', hue='PASS', data=subdf3[(subdf3['log2Frac']>0) & (subdf3['ANN_TYPE'].isin(['cassette']))])
plt.xlabel('Element')
plt.xticks(rotation=45, ha='right')
plt.ylabel('Fraction')
plt.title('C26 samples')
plt.legend(title="Passage")
#plt.legend(bbox_to_anchor=(1.05, 1))

plt.subplot(3,3, 9)
sns.swarmplot(x='AFF', y='FRAC', hue='IMPACT',data=subdf3[(subdf3['FRAC']>=1.5) & (subdf3['ANN_TYPE']=='cassette')  & (subdf3['IMPACT']!='MODIFIER')])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Element')
plt.ylabel('Fraction')
plt.legend(title="Impact")

plt.tight_layout()
plt.savefig('./results/supfigS2.svg')
plt.savefig('./results/supfigS2.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [109]:
data3[(data3['PASS']==15) & (data3['FRAC']>19) & (data3['ANN_TYPE']=='cassette')]

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
267963,p15_9_S7,15,580666,2.222470e-15,11,9,2,22.222221,T,C,missense_variant,MODERATE,cas2 cas9B,p.Asp39Gly,cassette


## 5. Exploration of the effect of the mutations explaining the escape rate (supplementary figure 3)

Integrating the previous results we will explore the mutations that could mainly explain the rate of cell escaping the killswitch circuit (we use the median value to filter out non-significant positions)

In [125]:
plt.figure(figsize=(10,5))
plt.subplot(1,3,1)
sns.swarmplot(x='AFF', y='FRAC', hue='IMPACT',data=subdf1[(subdf1['FRAC']>=1.5) & (subdf1['ANN_TYPE']=='cassette')  & (subdf1['IMPACT']!='MODIFIER')])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Element')
plt.ylabel('Fraction')

plt.subplot(1,3,2)
sns.swarmplot(x='AFF', y='FRAC', hue='IMPACT',data=subdf2[(subdf2['FRAC']>=1.5) & (subdf2['ANN_TYPE']=='cassette')  & (subdf2['IMPACT']!='MODIFIER')])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Element')
plt.ylabel('Fraction')

plt.subplot(1,3,3)
sns.swarmplot(x='AFF', y='FRAC', hue='IMPACT',data=subdf3[(subdf3['FRAC']>=1.5) & (subdf3['ANN_TYPE']=='cassette')  & (subdf3['IMPACT']!='MODIFIER')])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Element')
plt.ylabel('Fraction')
plt.tight_layout()
plt.savefig('./results/supfigS3.svg')
plt.savefig('./results/supfigS3.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [677]:
selected = data1[(data1['PASS']==18) & (data1['ANN_TYPE'].isin(['cassette']))].sort_values(['FRAC'])
selected['TOP'] = [1 if z>=1.5 else 0 for z in selected['FRAC']]
selected.to_excel('./results/suptableS2.xlsx')
selected

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE,TOP
443045,p3IPTG_A_S1,18,566372,2.184800e-15,677,676,1,0.147929,CTA,TTT,stop_gained,HIGH,LacI4,p.LeuGly102*,cassette,0
443032,p3IPTG_A_S1,18,566245,1.137610e-15,670,669,1,0.149477,G,T,missense_variant,MODERATE,LacI4,p.Ala145Asp,cassette,0
443043,p3IPTG_A_S1,18,566351,0.000000e+00,655,654,1,0.152905,T,C,missense_variant,MODERATE,LacI4,p.Met110Val,cassette,0
443077,p3IPTG_A_S1,18,566562,0.000000e+00,655,654,1,0.152905,T,G,synonymous_variant,LOW,LacI4,p.Ala39Ala,cassette,0
443082,p3IPTG_A_S1,18,566622,0.000000e+00,647,646,1,0.154799,A,C,stop_gained,HIGH,LacI4,p.Tyr19*,cassette,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494861,p3IPTG_A_S1b,18,566416,3.471600e-14,281,255,26,10.196078,GGTGCGT,CGTTCTT,missense_variant,MODERATE,LacI4,p.HisAlaPro86GlnGluArg,cassette,1
495059,p3IPTG_A_S1b,18,569127,8.076710e-15,235,210,25,11.904762,CAAACT,CAACT,frameshift_variant,HIGH,cas9B,p.Phe897fs,cassette,1
443366,p3IPTG_A_S1,18,569125,0.000000e+00,534,463,71,15.334773,ATCAAACT,GTCAACC,frameshift_variant&missense_variant,HIGH,cas9B,p.Lys896fs,cassette,1
494831,p3IPTG_A_S1b,18,566053,0.000000e+00,271,224,47,20.982143,CGCAA,CACAT,missense_variant,MODERATE,LacI4,p.LeuArg208MetCys,cassette,1


In [678]:
selected[selected['POS'].isin([566423, 571681, 566053])]

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE,TOP
443045,p3IPTG_A_S1,18,566372,2.184800e-15,677,676,1,0.147929,CTA,TTT,stop_gained,HIGH,LacI4,p.LeuGly102*,cassette,0
443032,p3IPTG_A_S1,18,566245,1.137610e-15,670,669,1,0.149477,G,T,missense_variant,MODERATE,LacI4,p.Ala145Asp,cassette,0
443043,p3IPTG_A_S1,18,566351,0.000000e+00,655,654,1,0.152905,T,C,missense_variant,MODERATE,LacI4,p.Met110Val,cassette,0
443077,p3IPTG_A_S1,18,566562,0.000000e+00,655,654,1,0.152905,T,G,synonymous_variant,LOW,LacI4,p.Ala39Ala,cassette,0
443082,p3IPTG_A_S1,18,566622,0.000000e+00,647,646,1,0.154799,A,C,stop_gained,HIGH,LacI4,p.Tyr19*,cassette,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494861,p3IPTG_A_S1b,18,566416,3.471600e-14,281,255,26,10.196078,GGTGCGT,CGTTCTT,missense_variant,MODERATE,LacI4,p.HisAlaPro86GlnGluArg,cassette,1
495059,p3IPTG_A_S1b,18,569127,8.076710e-15,235,210,25,11.904762,CAAACT,CAACT,frameshift_variant,HIGH,cas9B,p.Phe897fs,cassette,1
443366,p3IPTG_A_S1,18,569125,0.000000e+00,534,463,71,15.334773,ATCAAACT,GTCAACC,frameshift_variant&missense_variant,HIGH,cas9B,p.Lys896fs,cassette,1
494831,p3IPTG_A_S1b,18,566053,0.000000e+00,271,224,47,20.982143,CGCAA,CACAT,missense_variant,MODERATE,LacI4,p.LeuArg208MetCys,cassette,1


In [681]:
data2[(data2['ANN_TYPE']=='cassette') & (data2['FRAC']>10)]

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
54232,p2_6_S2,2,566587,3.889600e-06,4,3,1,33.333332,G,A,missense_variant,MODERATE,cas1 LacI4,p.Thr31Ile,cassette
163230,p2_6_S4,2,780968,4.857580e+03,156,1,155,15500.000000,AGTCC,CGTCC,upstream_gene_variant,MODIFIER,cas2 regulator gRNA5,c.-4520T>G,cassette
163253,p2_6_S4,2,781179,6.057650e-15,80,71,9,12.676056,GACCCAACTGCCACGAAGTTTT,GACCCAACTGCCTTGATGTTAT,upstream_gene_variant,MODIFIER,cas2 regulator gRNA2,c.-4751_-4743delAAACTTCGTinsTAACATCAA,cassette
163342,p2_6_S4,2,786151,5.734500e+03,182,1,181,18100.000000,TCT,TTT,synonymous_variant,LOW,cas2 cat,p.Val195Val,cassette
163362,p2_6_S4,2,786347,0.000000e+00,14,11,3,27.272728,A,G,missense_variant,MODERATE,cas2 lacI4,p.Tyr24Cys,cassette
225311,p3_B_S1,3,780968,6.689500e+03,214,2,212,10600.000000,AGTCC,CGTCC,upstream_gene_variant,MODIFIER,cas2 regulator gRNA5,c.-4520T>G,cassette
277012,p3_B_S2,3,780964,5.288950e+03,170,2,168,8400.000000,GGCTAGTCC,GGCTCGTCC,upstream_gene_variant,MODIFIER,cas2 regulator gRNA5,c.-4520T>G,cassette
277062,p3_B_S2,3,786151,7.229180e+03,228,1,227,22700.000000,TCTGTG,TTTGTG,synonymous_variant,LOW,cas2 cat,p.Val195Val,cassette
413818,p15_7_S5,15,780968,3.700950e+03,118,1,117,11700.000000,AGTCC,CGTCC,upstream_gene_variant,MODIFIER,cas2 regulator gRNA5,c.-4520T>G,cassette
413844,p15_7_S5,15,785418,4.709100e-15,7,6,1,16.666666,A,C,synonymous_variant,LOW,cas2 cas9B,p.Thr1339Thr,cassette


In [661]:
selected[selected['IMPACT']=='HIGH']

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
494840,p3IPTG_A_S1b,18,566136,0.000000e+00,243,238,5,2.100840,TAAGCGGGTCCCATCTTCGT,TTAGCCGGTCCCATCTTCGT,stop_gained,HIGH,LacI4,p.ArgLeu180*,cassette
494818,p3IPTG_A_S1b,18,565929,6.208090e-15,232,227,5,2.202643,AACAATCCCCTCATTTAA,AACAATCCCCTTTTTTTA,stop_gained,HIGH,LacI4,p.LeuAsnGlu245*,cassette
443663,p3IPTG_A_S1,18,571681,0.000000e+00,481,469,12,2.558635,ATTTTTTTTGATA,ATTTTTTTGATA,frameshift_variant,HIGH,cas9B,p.Asn46fs,cassette
495218,p3IPTG_A_S1b,18,571681,7.895900e-15,191,182,9,4.945055,ATTTTTTTTGATACT,ATTTCTTTAGAAACA,stop_gained,HIGH,cas9B,p.SerIleLysLys42*,cassette
495059,p3IPTG_A_S1b,18,569127,8.076710e-15,235,210,25,11.904762,CAAACT,CAACT,frameshift_variant,HIGH,cas9B,p.Phe897fs,cassette
443366,p3IPTG_A_S1,18,569125,0.000000e+00,534,463,71,15.334773,ATCAAACT,GTCAACC,frameshift_variant&missense_variant,HIGH,cas9B,p.Lys896fs,cassette
443009,p3IPTG_A_S1,18,566053,1.947940e-13,559,446,113,25.336323,CGCAAA,TGCTAT,stop_gained,HIGH,LacI4,p.LeuArg208*,cassette


# TOPLOT 

- C5 fractions versus percentage of variants per annotation type
- C5 cassette separating LacI4, Cas9B, gRNA/s only and impact+fraction

In [24]:
plt.figure()
c = 1
for pas in [2, 3, 15, 18]:
    plt.subplot(4,1, c)
    sns.scatterplot(x='POS', y='FRAC', hue='IMPACT', data=data1[(data1['FRAC']>2.5) & (data1['PASS']==pas) & (~data1['ANN_TYPE'].isin(['essential', 'non-essential']))])
    c+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
plt.figure()
c = 1
for pas in [2, 3, 15]:
    plt.subplot(4,1, c)
    sns.scatterplot(x='POS', y='FRAC', hue='IMPACT', data=data2[(data2['FRAC']>2.5) & (data2['FRAC']<100) & (data2['PASS']==pas) & (~data2['ANN_TYPE'].isin(['essential', 'non-essential']))])
    c+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## NOT USED

In [635]:
subdf1[(subdf1['FRAC']>=25) & (subdf1['ANN_TYPE']=='intergenic')]

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE,log2Frac,Condition,annotation
20557,p2_2_S1,2,629173,1.383230e-14,30,17,13,76.470589,GTTTTTTTTTTTTTTTTAGTTTGAAC,GTTTTTTTTTATTTATAAGTTTGAAC,upstream_gene_variant,MODIFIER,MPN508,c.-2605_-2599delAAAAAAAinsTATAAAT,intergenic,6.256833,2,intergenic
33962,p2_2_S2,2,113969,2.790510e-07,6,4,2,50.000000,T,G,upstream_gene_variant,MODIFIER,MPN094,c.-2292T>G,intergenic,5.643856,2,intergenic
35494,p2_2_S2,2,141270,4.924100e-14,59,40,19,47.500000,CAGAGAGAGAGAGAGAGAGAGC,CAGAGAGAGAGAGAGAGAGC,upstream_gene_variant,MODIFIER,MPN109,c.-624_-623delAG,intergenic,5.569856,2,intergenic
57248,p2_2_S2,2,528761,2.688110e-14,45,33,12,36.363636,GTTTTTTTTTTTTTTTTTGAAAGA,GTTTTTTTTTTTTTTTTGAAAGA,upstream_gene_variant,MODIFIER,MPN435,c.-3956delA,intergenic,5.184424,2,intergenic
63072,p2_2_S2,2,629170,2.953720e-15,57,38,19,50.000000,CTAGTTTTTTTTTTTTTTTTAGTTTGAAC,TTAGATTTTTTTTTTTTTTTTAGTTTGAAC,upstream_gene_variant,MODIFIER,MPN508,c.-2614_-2586delGTTCAAACTAAAAAAAAAAAAAAAACTAGi...,intergenic,5.643856,2,intergenic
85859,p2_2_S3,2,141267,1.219870e-13,159,97,62,63.917526,TCTCAGAGAGAGAGAGAGAGAGAGC,ACTCAGAGAGAGAGAGAGAGAGAGAGC,upstream_gene_variant,MODIFIER,MPN109,c.-646_-622delTCTCAGAGAGAGAGAGAGAGAGAGCinsACTC...,intergenic,5.998140,2,intergenic
161975,p3_D_S2,3,195424,4.551630e-13,413,239,174,72.803345,TTTCCAAAAAAAAAAAAAAAGTAAAATAGAAAAGC,ATTCTGAAAAAAAAAAAAAAAAGTAAAATAGAAAAGC,upstream_gene_variant,MODIFIER,MPN148,c.-97_-63delTTTCCAAAAAAAAAAAAAAAGTAAAATAGAAAAG...,intergenic,6.185933,3,intergenic
214268,p3_D_S2,3,629169,2.384300e-13,355,212,143,67.452827,TCTAGTTTTTTTTTTTTTTTTAGTTTGAAC,TCTAGTTTTTTTTTTTTCTATAGTTTGAAC,upstream_gene_variant,MODIFIER,MPN508,c.-2604_-2602delAAAinsTAG,intergenic,6.075807,3,intergenic
274642,p3_D_S3,3,528755,8.763400e-14,140,84,56,66.666664,TCAAACGTTTTTTTTTTTTTTTTTGAAAGAAATTGATTGCT,CCAATCGTTTTTTTTTTTTTTTTTGAAAGAAATTGATTGCT,upstream_gene_variant,MODIFIER,MPN435,c.-3937_-3933delTTTGAinsATTGG,intergenic,6.058894,3,intergenic
280252,p3_D_S3,3,618608,8.492520e-09,7,5,2,40.000000,A,G,upstream_gene_variant,MODIFIER,MPN504,c.-702A>G,intergenic,5.321928,3,intergenic


In [632]:
# For genes
plt.figure()
plt.subplot(2,1,1)
sns.swarmplot(x='IMPACT', y='FRAC', hue='PASS', data=subdf1[(subdf1['FRAC']>=5) & (subdf1['ANN_TYPE']=='gene')])
plt.xticks(rotation=45, ha='right')

plt.subplot(2,1,2)
sns.swarmplot(x='IMPACT', y='FRAC', hue='PASS',data=subdf2[(subdf2['FRAC']>=5) & (subdf2['ANN_TYPE']=='gene')])
plt.xticks(rotation=45, ha='right')


# For intergenic
plt.figure()
plt.subplot(2,1,1)
sns.swarmplot(x='IMPACT', y='FRAC', hue='PASS',data=subdf1[(subdf1['FRAC']>=5) & (subdf1['ANN_TYPE']=='intergenic')])
plt.xticks(rotation=45, ha='right')

plt.subplot(2,1,2)
sns.swarmplot(x='IMPACT', y='FRAC', hue='PASS',data=subdf2[(subdf2['FRAC']>=5) & (subdf2['ANN_TYPE']=='intergenic')])
plt.xticks(rotation=45, ha='right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([0, 1]), <a list of 2 Text xticklabel objects>)

In [288]:
def plot_fixation(df):
    """ Plot to show the percentage of variants mapping to each type of annotation """
    colord = {'intergenic':0, 'gene':1, 'essential':2, 'non-essential':3, 'cassette':4}
    pasind = {2:0, 3:1, 15:2, 18:3}
    rs = {}
    colors = []
    for k, v in Counter(list(df['POS'])).items():
        if v>2:
            rs[k] = [0.0,0.0,0.0,0.0]
            subdf = df[df['POS']==k]
            for passage in set(subdf['PASS']):
                rs[k][pasind[passage]] = list(subdf[subdf['PASS']==passage]['FRAC'])
    return rs
dic  = plot_fixation(data1)

In [327]:
rs = {}
columns = ['p2_2_S1', 'p2_2_S2', 'p2_2_S3', 'p3_D_S2', 'p3_D_S3', 'p15_F_S1', 'p15_F_S5',  'p3IPTG_A_S1', 'p3IPTG_A_S1b']
for n in columns:
    rs[n] = []
    for m in columns:
        rs[n].append(round(mannwhitneyu(data1[(data1['SAMPLE']==n) & (data1['FRAC']>=1)]['FRAC'], data1[(data1['SAMPLE']==m) & (data1['FRAC']>=1)]['FRAC'])[1], 8))
pvalues = pd.DataFrame.from_dict(rs, orient='index')
pvalues.columns=columns
pvalues
        

,p2_2_S1,p2_2_S2,p2_2_S3,p3_D_S2,p3_D_S3,p15_F_S1,p15_F_S5,p3IPTG_A_S1,p3IPTG_A_S1b
p2_2_S1,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
p2_2_S2,0.0,0.500000,0.000000,0.000000,0.000000,0.304095,0.000000,0.000000,0.000000
p2_2_S3,0.0,0.000000,0.499998,0.000000,0.368914,0.000000,0.000000,0.297108,0.000005
p3_D_S2,0.0,0.000000,0.000000,0.499997,0.000000,0.000000,0.481911,0.000000,0.000000
p3_D_S3,0.0,0.000000,0.368914,0.000000,0.499996,0.000000,0.000000,0.370263,0.000963
p15_F_S1,0.0,0.304095,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000
p15_F_S5,0.0,0.000000,0.000000,0.481911,0.000000,0.000000,0.499997,0.000000,0.000000
p3IPTG_A_S1,0.0,0.000000,0.297108,0.000000,0.370263,0.000000,0.000000,0.499991,0.008616
p3IPTG_A_S1b,0.0,0.000000,0.000005,0.000000,0.000963,0.000000,0.000000,0.008616,0.499999


In [352]:
def extracta(df, sample):
    rs = []
    subdf = df[df['SAMPLE']==sample].copy()
    fractions = {}
    for k, v in zip(subdf['POS'], subdf['FRAC']):
        if k in fractions:
            fractions[k].append(v)
        else:
            fractions[k] = [v]
    fractions = {k:np.mean(v) for k, v in fractions.items()}
    for i in range(1, len(genome_C5)+1):
        if i in fractions:
            if fractions[i]>=1:
                rs.append(i)
            else:
                rs.append(1)
    return np.array(rs)

def volcano():
    rs = {}
    for sample in set(data1[data1['PASS'].isin([3,18])]['SAMPLE']):
         rs[sample] = extracta(data1, sample)
    fcs = []
    pvals = []
    positions = []
    for i in range(1, len(genome_C5)+1):
        try:
            vals = np.array([rs['p3IPTG_A_S1b'][i], rs['p3IPTG_A_S1'][i], rs['p3_D_S3'][i], rs['p3_D_S2'][i]])
            if len(vals[vals>1])==0:
                pass
            else:
                fc = np.log2(np.mean([rs['p3IPTG_A_S1b'][i], rs['p3IPTG_A_S1'][i]]))-np.log2(np.mean([rs['p3_D_S3'][i], rs['p3_D_S2'][i]]))
                pval = -1*np.log10(mannwhitneyu([rs['p3IPTG_A_S1b'][i], rs['p3IPTG_A_S1'][i]], [rs['p3_D_S3'][i], rs['p3_D_S2'][i]])[1])
                fcs.append(fc)
                pvals.append(pval)
                positions.append(i)
        except:
            pass
    return fcs, pvals, positions
            

In [353]:
f, pv, p  = volcano()

In [357]:
plt.figure()
plt.scatter(f, pv)
plt.axhline(y=-1*np.log10(0.05), linestyle='--', color='r')

/home/smiravet/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [601]:
background = np.log2(data1[(data1['PASS']==3) & (data1['FRAC']>=1)]['FRAC'])
plt.figure()
sns.distplot(background)
plt.axvline(x=np.mean(background))
plt.axvline(x=np.percentile(background,99), color='r', linestyle='--')
selected = data1[data1['PASS']==18].copy()
#selected['pvalue'] = [mannwhitneyu(frac, background) for frac in list(selected['FRAC'])]

/home/smiravet/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
plt.figure()
c = 1
for pas in [2, 3, 15, 18]:
    plt.subplot(4,1, c)
    sns.scatterplot(x='POS', y='FRAC', hue='IMPACT', data=data1[(data1['FRAC']>2.5) & (data1['PASS']==pas) & (~data1['ANN_TYPE'].isin(['essential', 'non-essential']))])
    c+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
data1[(data1['PASS']==18) & (data1['ANN_TYPE']!='cassette') & (data1['FRAC']>5)  & (data1['IMPACT']!='MODIFIER')].sort_values('FRAC')

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
405668,p3IPTG_A_S1,18,130837,6.795500e-15,41,39,2,5.128205,A,G,missense_variant,MODERATE,MPN101,p.Ser133Gly,gene
496576,p3IPTG_A_S1b,18,602108,0.000000e+00,40,38,2,5.263158,CAACTGTAA,TAACAGTAA,missense_variant,MODERATE,MPN489,p.Gln254Leu,gene
496332,p3IPTG_A_S1b,18,595882,4.776910e-15,20,19,1,5.263158,C,T,missense_variant,MODERATE,MPN485,p.Gly389Arg,gene
446239,p3IPTG_A_S1,18,602189,0.000000e+00,277,263,14,5.323194,TTGGGTGAGGGGGAGGGGGTTT,TAGGGAGAGGGGGAGGGGGTTT,missense_variant,MODERATE,MPN489,p.ThrGln227SerLeu,gene
481872,p3IPTG_A_S1b,18,284317,5.728050e-16,158,150,8,5.333333,C,T,synonymous_variant,LOW,MPN233,p.Leu418Leu,gene
412492,p3IPTG_A_S1,18,207198,0.000000e+00,291,276,15,5.434783,AGTACCACCACCA,GGGACCTCCGCCG,missense_variant,MODERATE,MPN154,p.Val478Gly,gene
446222,p3IPTG_A_S1,18,602038,0.000000e+00,58,55,3,5.454545,T,A,missense_variant,MODERATE,MPN489,p.Thr279Ser,gene
489010,p3IPTG_A_S1b,18,442873,1.190810e-15,115,109,6,5.504587,GTGCTTAGTGGT,GAGCTAAGAGGA,stop_gained,HIGH,MPN370,p.CysLeuValVal542*,gene
484509,p3IPTG_A_S1b,18,340718,0.000000e+00,19,18,1,5.555555,C,T,synonymous_variant,LOW,MPN285,p.Leu170Leu,gene
496334,p3IPTG_A_S1b,18,596162,0.000000e+00,19,18,1,5.555555,T,C,synonymous_variant,LOW,MPN485,p.Thr295Thr,gene


In [250]:
new_dic = {}
for k, v in dic.items():
    a = np.array(v)
    try:
        if len(a[1])==2 and len(a[-1])==2:
            new_dic[k] = a
    except:
        pass

In [270]:
x3, x3i, x15 = [[], [], []]
positions = []
for k, v in dic.items():
    a = np.array(v)
    try:
        if len(a[0])==2:
            if len(a[1])==2:
                x3.append(np.mean(a[1])/np.mean(a[0]))
            elif len(a[2])==2:
                x15.append(np.mean(a[2])/np.mean(a[0]))
            elif len(a[-1])==2:
                positions.append(k)
                x3i.append(np.mean(a[-1])/np.mean(a[0]))
            else:
                pass
    except:
        pass

In [271]:
plt.figure()
plt.violinplot([x3, x15, x3i])
plt.xticks([1,2,3],['P3/P2', 'P15/P2', 'P3-IPTG/P2'])
plt.axhline(y=np.mean(x3)+2*np.std(x3), color='red', linestyle='--', label='95% threshold P3/P2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [274]:
thr = np.mean(x3)+2*np.std(x3)
thr

1.0411993871078709

In [279]:
a = np.array(x3i)
b = np.array(positions)

In [282]:
b[a>=thr]

array([211633, 264869, 198159, 224676, 254989, 458314, 553466, 675401,
       767918, 807060])

## 3. Fixation of the mutations 

In this section we evaluate which mutations increase their representation from passage 2 to 3 and comparing this ratio between non-treated and induced with IPTG samples. 

In [115]:
data1[(data1['ALTN']>10) & (data1['PASS']==3)]

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
137985,p3_D_S2,3,6772,1.989160e-15,582,569,13,2.284710,TCTTTAAGGAAAA,ACTTAAAGGAAAA,missense_variant,MODERATE,MPN004,p.LeuPhe651HisLeu,gene
138227,p3_D_S2,3,8563,0.000000e+00,523,512,11,2.148438,AAACCAAAATAAACCAATGAAACAAGGCGTGT,AAACCAAAATAATCTATTGAAACAAGGCGTGT,initiator_codon_variant,LOW,MPN006,p.Met1?,gene
139095,p3_D_S2,3,15285,4.221290e-16,684,673,11,1.634472,CCCTAAAGAACAAGG,TCCTGAAGAACAAGG,missense_variant,MODERATE,MPN013,p.Lys100Glu,gene
139787,p3_D_S2,3,20319,2.260140e-14,599,588,11,1.870748,CCCGTTCCCGCGTG,CCCGTTCCCGAGAG,missense_variant,MODERATE,MPN018,p.Val336Glu,gene
140828,p3_D_S2,3,28083,0.000000e+00,644,633,11,1.737757,AATCCACGGCCGTTT,AATCCACGGCTGTCT,missense_variant,MODERATE,MPN022,p.ArgPhe260CysLeu,gene
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267794,p3_D_S3,3,414743,2.170210e-15,172,161,11,6.832298,T,A,synonymous_variant,LOW,MPN347,p.Gly268Gly,gene
269396,p3_D_S3,3,442878,0.000000e+00,235,222,13,5.855856,TAGTGGT,TGGAGGT,missense_variant,MODERATE,MPN370,p.Val544Glu,gene
274642,p3_D_S3,3,528755,8.763400e-14,140,84,56,66.666664,TCAAACGTTTTTTTTTTTTTTTTTGAAAGAAATTGATTGCT,CCAATCGTTTTTTTTTTTTTTTTTGAAAGAAATTGATTGCT,upstream_gene_variant,MODIFIER,MPN435,c.-3937_-3933delTTTGAinsATTGG,intergenic
280845,p3_D_S3,3,629170,0.000000e+00,175,104,71,68.269234,CTAGTTTTTTTTTTTTTTTTAGTTTGAACTCA,TTAGCTTTTTTTTTTTTTTTTAGTTTGAACTCA,upstream_gene_variant,MODIFIER,MPN508,c.-2617_-2586delTGAGTTCAAACTAAAAAAAAAAAAAAAACT...,intergenic


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
def plot_fixation(df):
    
    subdf = df.groupby(['PASS', 'POS', 'ALT']).sum()
    return subdf

    positions = set([k for k, v in Counter(subdf['POS']).items() if v>=3])
    return df[df['POS'].isin(positions)]

In [57]:
np.array([i for i in plot_fixation(snpcalls1).index])

array([['2', '6', 'T'],
       ['2', '13', 'C'],
       ['2', '31', 'A'],
       ...,
       ['18', '822913', 'TGCGTGAAA'],
       ['18', '822925', 'C'],
       ['18', '822931', 'CAAGTTCT']], dtype='<U73')

In [196]:
plt.figure(figsize=(10,10))
sns.scatterplot(x='POS', y='FRAC', hue='PASS', data=effect1[(effect1['IMPACT']=='HIGH') & (effect1['FRAC']>=1)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
plt.figure()
effect21['logFRAC'] = np.log2(effect21['FRAC'])
sns.violinplot(x='IMPACT', y='FRAC', hue='PASS', data=effect1)

/home/smiravet/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
snpcalls1[snpcalls1['SAMPLE']=='p2_2_S1'].sort_values('FRAC')

,POS,REF,ALT,QUAL,TOT,REFN,ALTN,ALTS,FRAC,SAMPLE,P,EFF
18482,566325,C,T,1.469330e-15,205,204,1,1,0.487805,p2_2_S1,2,T|synonymous_variant|LOW|Gene_565549_566677|Ge...
18481,566311,A,G,7.046470e-16,198,197,1,1,0.505051,p2_2_S1,2,G|missense_variant|MODERATE|Gene_565549_566677...
18485,566383,GCC,CCT,0.000000e+00,181,180,1,1,0.552486,p2_2_S1,2,CCT|missense_variant|MODERATE|Gene_565549_5666...
18483,566361,A,G,0.000000e+00,172,171,1,1,0.581395,p2_2_S1,2,G|synonymous_variant|LOW|Gene_565549_566677|Ge...
18476,566244,A,G,1.167980e-15,171,170,1,1,0.584795,p2_2_S1,2,G|synonymous_variant|LOW|Gene_565549_566677|Ge...
...,...,...,...,...,...,...,...,...,...,...,...,...
6416,195428,CAAAAAAAAAAAAAAAGTAAAATAGAAAAGC,"CAAAATAAGAAAAAAAGTAAAATAGAAAAGC,CAAAAAAAAAAAAA...",0.000000e+00,24,15,9,"1,1,1,6",37.500000,p2_2_S1,2,CAAAATAAGAAAAAAAGTAAAATAGAAAAGC|upstream_gene_...
17103,528761,GTTTTTTTTTTTTTTTTTGAAAGA,"GTTTTTTTTTTTTTTTTGAAAGA,GTTTTTTTTTTTTTTTTTTGAAAGA",0.000000e+00,28,16,12,"6,6",42.857143,p2_2_S1,2,GTTTTTTTTTTTTTTTTGAAAGA|upstream_gene_variant|...
20563,629173,GTTTTTTTTTTTTTTTTAGTTTGAAC,"GTTTTTTTTTATTTATAAGTTTGAAC,GTTTTTTTTTTTTTTTAGT...",1.383230e-14,30,17,13,"1,2,6,2,2",43.333333,p2_2_S1,2,GTTTTTTTTTATTTATAAGTTTGAAC|upstream_gene_varia...
4753,141270,CAGAGAGAGAGAGAGAGAGAGC,"CAGAGAGAGAGAGAGAGAGC,CAGAGAGAGAGAGAGAGAGAGAGC",0.000000e+00,52,27,25,"7,18",48.076923,p2_2_S1,2,CAGAGAGAGAGAGAGAGAGC|upstream_gene_variant|MOD...


In [47]:
simplify_effect(snpcalls1, genome=20)

UnboundLocalError: local variable 'annotations' referenced before assignment

In [45]:
# Figure


def plot_analysis(df):
    repro = [k for k, v in Counter(list(df.POS)).items() if v>=2]
    plt.figure()
    plt.subplot(3,3,1)
    sns.lineplot(x='P', y='FRAC', data=df[(df['POS'].isin(posE)) & (df['POS'].isin(repro))], label='E')
    sns.lineplot(x='P', y='FRAC', data=df[(df['POS'].isin(posN)) & (df['POS'].isin(repro))], label='NE')
    sns.lineplot(x='P', y='FRAC', data=df[(df['POS'].isin(cas1)) & (df['POS'].isin(repro))], label='C')


plot_analysis(snpcalls1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 4. Substituion rate per base comparing cassette versus other distributions

To evaluate the selection in the cassette we compare the rate of substitutions per base occuring within the cassette, a set of essential and non-essential genes and with the general distribution observed at genome level. In this case we do not care about the fraction each variant is found but how many times a variant is found in the cassette versus the other distributions.

In [11]:
# Show the ratio of mutation in cassette versus general
def mutation_rate(snpdf, segment, mintot=5, minrefn=3, minaltn=2):
    _df = snpdf[(snpdf['TOT']>=mintot) & (snpdf['REFN']>=minrefn) & (snpdf['ALTN']>=minaltn)].copy()    
    rs = {}
    for sample in set(_df.SAMPLE):
        total_len = 816394
        _df2 = _df[_df['SAMPLE']==sample].copy()
        NE = _df2[_df2['POS'].isin(posN)]
        ES = _df2[_df2['POS'].isin(posE)]
        TG = _df2[~_df2['POS'].isin(segment)]
        CS = _df2[_df2['POS'].isin(segment)]
        #total_len+=len(segment)
        rs[sample] = [sample, sample.split('_')[0],
                      int(sample.split('_')[0].replace('p', '').replace('3IPTG', '18')),
                      TG.shape[0], ES.shape[0], NE.shape[0], CS.shape[0], 
                      sum(TG['FRAC']), sum(ES['FRAC']), sum(NE['FRAC']), sum(CS['FRAC'])]
    jj = pd.DataFrame.from_dict(rs, orient='index')
    jj.columns = ['SAMPLE', 'COND', 'P', 'T', 'E', 'N', 'C', 'TF', 'EF', 'NF', 'CF']
    jj['T%'] = 100*jj['T']/(total_len)  
    jj['E%'] = 100*jj['E']/len(posE)
    jj['N%'] = 100*jj['N']/len(posN)
    jj['C%'] = 100*jj['C']/len(segment)
    return jj

def tp(df, order):
    rs = {}
    c=0
    for row, col in df.iterrows():
        rs[c] = [col['COND'],'Essential', col['E%'], col['EF']]
        rs[c+1] = [col['COND'],'Non-essential', col['N%'], col['NF']]
        rs[c+2] = [col['COND'],'Chromosome', col['T%'], col['TF']]
        rs[c+3] = [col['COND'],'Cassette', col['C%'], col['CF']]
        c+=4
    a = pd.DataFrame.from_dict(rs, orient='index')
    a.columns = ['Condition', 'Loci', 'Variation per base [%]', 'Accumulated Fraction [%]']
    return orderdf(a, ordered_classes=order, col='Condition')

In [32]:
mutrate11 = mutation_rate(snpcalls1, segment=cas1)
mutrate21 = mutation_rate(snpcalls2, segment=cas1)
mutrate22 = mutation_rate(snpcalls2, segment=cas2)
mutrate212 = mutation_rate(snpcalls2, segment=cas12)

In [33]:
plt.close('all')
plt.figure(figsize=(15, 3))
c = 1
for text, df in zip(['C5 Par', 'C20 Par', 'C20 3b'], [mutrate11, mutrate21, mutrate22]):
    plt.subplot(1,3,c)
    if text=='C5 Par':
        sns.barplot(x='Condition', y='Variation per base [%]', hue='Loci', palette='mako', data=tp(df, ['p2','p3','p15', 'p3IPTG']))
    else:
        sns.barplot(x='Condition', y='Variation per base [%]', hue='Loci', palette='mako', data=tp(df, ['p2','p3','p15']))
    plt.ylim(0, 15)
    c+=1
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
from dnds_functions import evolution_of_sample, dnds
ttt = evolution_of_sample(effect1[effect1['IMPACT']=='HIGH'])

In [40]:
effectC= simplify_effect(snpcalls1, cas1, minaltn=3)
effectE= simplify_effect(snpcalls1, posE, minaltn=3)
effectN= simplify_effect(snpcalls1, posN, minaltn=3)

tttC = evolution_of_sample(effectC)
tttE = evolution_of_sample(effectE)
tttN = evolution_of_sample(effectN)

In [53]:
tttE

{'p15_F_S1': [0.015,
  0.011,
  1.3636363636363638,
  0.015,
  0.011,
  1.3636363636363638],
 'p3IPTG_A_S1': [0.418,
  0.265,
  1.5773584905660376,
  0.612,
  0.327,
  1.8715596330275228],
 'p3_D_S3': [0.012, 0.0, '+', 0.012, 0.0, '+'],
 'p2_2_S1': [0, 0, '/', 0, 0, '/'],
 'p3IPTG_A_S1b': [0, 0, '/', 0, 0, '/'],
 'p15_F_S5': [0, 0, '/', 0, 0, '/'],
 'p2_2_S3': [0.009000000000000001,
  0.022,
  0.40909090909090917,
  0.009000000000000001,
  0.022,
  0.40909090909090917],
 'p2_2_S2': [0.006, 0.0, '+', 0.006, 0.0, '+'],
 'p3_D_S2': [0.03,
  0.06599999999999999,
  0.4545454545454546,
  0.03,
  0.06599999999999999,
  0.4545454545454546]}

In [55]:
plt.figure()
x, y = [[], []]
for k, v in tttC.items():
    if type(v[-1])==float and type(tttE[k][-1])==float:
        x.append(tttE[k][-1])
        y.append(v[-1])
plt.scatter(x, y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [307]:
plt.figure()
plt.subplot(2,2,1)
sns.countplot(x='SAMPLE', data=snpcalls1)
plt.xticks(rotation=45, ha='right')
plt.title('Variant count per sample')

plt.subplot(2,2,2)
sns.boxplot(x='SAMPLE', y='FRAC', data=snpcalls1)
plt.xticks(rotation=45, ha='right')
plt.title('Fraction distribution per sample')

plt.subplot(2,2,3)
sns.countplot(x='SAMPLE', data=snpcalls2)
plt.xticks(rotation=45, ha='right')
plt.title('Variant count per sample')

plt.subplot(2,2,4)
sns.boxplot(x='SAMPLE', y='FRAC', data=snpcalls2)
plt.xticks(rotation=45, ha='right')
plt.title('Fraction distribution per sample')

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

General exploration of the most representative variants:

In [308]:

plt.figure(figsize=(15, 10))
plt.subplot(3,1,1)
sns.scatterplot(x='POS', y='FRAC', hue='SAMPLE', data=snpcalls1[(snpcalls1['FRAC']>=5) & (snpcalls1['TOT']>=10) & (snpcalls1['P']<=15) & (snpcalls1['FRAC']<100)])
plt.axvspan(565510, 572114, color='gray', alpha=0.2)
plt.xlabel('')
plt.ylabel('Fraction of variant reads [%]')
plt.xlim(1, 890000)

plt.subplot(3,1,2)
sns.scatterplot(x='POS', y='FRAC', hue='SAMPLE', data=snpcalls1[(snpcalls1['FRAC']>=5) & (snpcalls1['TOT']>=10) & (snpcalls1['P']>15) & (snpcalls1['FRAC']<100)])
plt.axvspan(565510, 572114, color='gray', alpha=0.2)
plt.xlabel('')
plt.ylabel('Fraction of variant reads [%]')
plt.xlim(1, 890000)

plt.subplot(3,1,3)
sns.scatterplot(x='POS', y='FRAC', hue='SAMPLE', data=snpcalls2[(snpcalls2['FRAC']>=5) & (snpcalls2['TOT']>=10) & (snpcalls2['FRAC']<100)])
plt.axvspan(cass_par[0], cass_par[1], color='gray', alpha=0.2)
plt.axvspan(cass_3b[0], cass_3b[1], color='gray', alpha=0.2)
plt.xlabel('genome position [bp]')
plt.ylabel('Fraction of variant reads [%]')
plt.xlim(1, 890000)

plt.savefig('./figures/supfig1.svg')
plt.savefig('./figures/supfig1.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Selection of variants
In this case we evaluate the fraction of each variant and how it is fixed along passages. As not all the samples are directly related, we consider the induced sample (IPTG) as 18. In this sense, if a mutation found in passages 2, 3 or 15 is found in 18 with a higher rate it implies that this variants are able to confere the capability to surpass the killswitch

In [610]:
def plot_evolution(df):
    positions = set(df[df['FRAC']>1].POS)
    plt.figure()
    plt.subplot(1,2,1)
    sns.lineplot(x='PASS', y='FRAC', hue='POS', data=df[df['POS'].isin(positions)])
    plt.subplot(1,2,2)
    sns.boxplot(x='PASS', y='FRAC', data=df[df['POS'].isin(positions)])


In [611]:
plot_evolution(data1)


/home/smiravet/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyboardInterrupt: 

In [182]:
annotations

{'LacI4': [565549, 566678, '-'],
 'pS': [566678, 566721, '-'],
 'Par cassette': [566747, 567674, '-'],
 'cas9B': [567711, 571818, '-'],
 'pG64': [571818, 571934, '-'],
 'gRNA2': [571942, 572044, '-'],
 'protospacer12 (10 targets)': [572024, 572044, '-'],
 'p438': [572044, 572066, '-'],
 'IR-OR': [572088, 572114, '-'],
 'MTn insertion point': [572113, 572124, '-']}

In [330]:
from Bio.Seq import Seq
genes_affected = {}
for k, v in annotations.items():
    if k in ['cas9B', 'LacI4']:
        if k=='cas9B':
            seq = Seq(genome_C5[v[0]:v[1]])
        else:
            seq = Seq(genome_C5[v[0]:v[1]])
        print(len(seq)/3)
        rvc = seq.reverse_complement()
        mrn = rvc.transcribe()
        prt = mrn.translate(table=4)
        genes_affected[k] = [seq, rvc, mrn, prt, len(prt)]

376.3333333333333
1369.0


/home/smiravet/.local/lib/python3.6/site-packages/Bio/Seq.py:2309: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [333]:
genes_affected['LacI4'][3][-10:]

Seq('RLESGQ*R*V', HasStopCodon(ExtendedIUPACProtein(), '*'))

LacI4 has extra bases in the annotation, that makes it to be non-codonic. I will consider the first stop found (move the end 10 bases)...

In [33]:
from Bio.Seq import Seq
genes_affected = {}
for k, v in annotations.items():
    if k in ['cas9B', 'LacI4']:
        if k=='cas9B':
            seq = Seq(genome_seq[v[0]:v[1]])
        else:
            seq = Seq(genome_seq[v[0]+10:v[1]])
        print(len(seq)/3)
        rvc = seq.reverse_complement()
        mrn = rvc.transcribe()
        prt = mrn.translate(table=4)
        genes_affected[k] = [seq, rvc, mrn, prt, len(prt)]

NameError: name 'annotations' is not defined

In [34]:
genes_affected['LacI4'][3][-10:]

KeyError: 'LacI4'

In [ ]:
aa

# Evaluate effect of the selected mutations

In [35]:
snpcalls1

,POS,REF,ALT,QUAL,TOT,REFN,ALTN,FRAC,SAMPLE,P,EFF
0,70,C,T,4.927350e-15,60,58,2,3.333333,p2_2_S1,2,T|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
1,149,T,C,0.000000e+00,75,74,1,1.333333,p2_2_S1,2,C|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
2,150,A,T,2.263910e-15,69,68,1,1.449275,p2_2_S1,2,T|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
3,154,T,C,0.000000e+00,62,61,1,1.612903,p2_2_S1,2,C|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
4,176,TAAT,CAAC,3.420820e-15,58,57,1,1.724138,p2_2_S1,2,CAAC|upstream_gene_variant|MODIFIER|dnaN|MPN00...
...,...,...,...,...,...,...,...,...,...,...,...
41166,822775,AACGT,GACGC,3.631530e-15,171,170,1,0.584795,p3IPTG_A_S1b,18,GACGC|missense_variant|MODERATE|soj|MPN688|tra...
41167,822787,A,G,1.152910e-15,175,173,2,1.142857,p3IPTG_A_S1b,18,G|synonymous_variant|LOW|soj|MPN688|transcript...
41168,822830,T,G,3.857310e-15,165,164,1,0.606061,p3IPTG_A_S1b,18,G|missense_variant|MODERATE|soj|MPN688|transcr...
41169,822867,TTTTT,CTTTC,0.000000e+00,172,171,1,0.581395,p3IPTG_A_S1b,18,CTTTC|missense_variant|MODERATE|soj|MPN688|tra...


NameError: name 'selected' is not defined

In [184]:
selected

NameError: name 'selected' is not defined

In [38]:
selected_ann

NameError: name 'selected_ann' is not defined

In [39]:
selected_ann[selected_ann['IMPACT']=='HIGH']

NameError: name 'selected_ann' is not defined

# Mutation rate study



In [92]:
plt.close('all')

In [44]:
pd.melt(mutrate1.sort_values('P')[[i for i in mutrate1.columns if '%' in i]])

NameError: name 'mutrate1' is not defined

In [45]:
plt.figure()
plt.boxplot([mutrate1['N%'], mutrate2['N%']])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'mutrate1' is not defined

In [46]:
plt.close('all')
plt.figure()
x, y = ['OUT%', 'IN%']
plt.plot([0,16],[0,16], c='grey', linestyle='--')
sns.scatterplot(x=x, y=y, data=mutrate1)
for i in range(mutrate1.shape[0]):
    plt.text(x=mutrate1[x][i]+0.2,y=mutrate1[y][i]+0.3,s=mutrate1.index[i],
             fontdict=dict(size=8))
sns.scatterplot(x=x, y=y, data=mutrate2)
c=0
for i in range(mutrate2.shape[0]):

    if c==0:
        plt.text(x=mutrate2[x][i]+0.2,y=mutrate2[y][i]+0.4,s=mutrate2.index[i],
                 fontdict=dict(size=10))
        c==1
    else:
        plt.text(x=mutrate2[x][i]+0.2,y=mutrate2[y][i]+0.2,s=mutrate2.index[i],
                 fontdict=dict(size=8))
        c==0
plt.xlabel('Genome Variant Rate [%]')
plt.ylabel('Cassette Variant Rate [%]')
plt.xlim(0,16)
plt.ylim(0,16)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'mutrate1' is not defined

In [47]:
from scipy.stats import wilcoxon, ttest_rel
from collections import Counter

wrs = {}
trs = {}
ns = dict(Counter(mutrate.P))
for p in set(mutrate.P):
    wrs[p] = wilcoxon(x=list(mutrate[mutrate['P']==p]['OUT%']),
                      y=list(mutrate[mutrate['P']==p]['IN%']))[1]
    trs[p] = ttest_rel(list(mutrate[mutrate['P']==p]['OUT%']),
                       list(mutrate[mutrate['P']==p]['IN%']))[1]

NameError: name 'mutrate' is not defined

In [48]:
ns, wrs, trs

NameError: name 'ns' is not defined

null hypothesis (same mut rate) cannot be rejected at a confidence level of 5%, only at early passages 2 and 3 seems to be limited

In [49]:
snpcalls50

NameError: name 'snpcalls50' is not defined

# Plot variations

In [50]:
cass_par

[565510, 572124]

In [51]:
selected

NameError: name 'selected' is not defined

In [52]:

from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram


def plot_variations(genbank_file, coords, variations):
    record = SeqIO.read(genbank_file, "genbank")

    gd_diagram = GenomeDiagram.Diagram(record.id)
    gd_track_for_features = gd_diagram.new_track(1, name="Annotated Features")
    gd_feature_set = gd_track_for_features.new_set()

    for feature in record.features:
        if feature.location.start>=coords[0] and feature.location.start<=coords[1] and feature.type!='Polymorphism':
            print(feature.type, feature.qualifiers['label'], feature.location)
            feature.location.strand=-1
            #if feature.type != "gene":
                # Exclude this feature
            #    continue
            #if len(gd_feature_set) % 2 == 0:
            #    color = colors.blue
            if feature.type=='CDS':
                color = colors.lightblue
                gd_feature_set.add_feature(
                    feature, sigil="ARROW", arrowshaft_height=1.0, color=color, height=0.2, label=True, label_size=14, label_angle=0
                )
            # else:
            #     color = colors.lightblue
            #     gd_feature_set.add_feature(
            #         feature, sigil="BOX", color=color, height=0.1, label=True, label_size=14, label_angle=10)
    colordic = {'stop': colors.firebrick, 'frameshift':colors.red, 'missense':colors.orange}
    done = []

    for pos, eff in zip(variations.POS, variations.EFF):
        if pos not in done:
            effi = eff.split('|')[1].split('_')[0]
            color = colordic[effi]
            feature = SeqFeature(FeatureLocation(pos, pos+1))
            gd_feature_set.add_feature(
                    feature,
                    color=color,
                    name=' '+effi,
                    label=True,
                    label_size=15,
                    size=20
                    #label_color=color,
                )
            done.append(pos)
            for i in range(pos-30, pos+30):
                done.append(i)
    gd_diagram.draw(format="linear", pagesize="A5", fragments=1, start=565510, end=572124)
    gd_diagram.write("plasmid_linear_nice.svg", "SVG")

In [53]:
plot_variations('./data/C5_refSeq.gb', coords=[565549,571818], variations=selected)

NameError: name 'selected' is not defined

In [637]:
raw_seq, new_seq = get_new_sequence(seq_record5, 'LacI4', pos=[565576, 'AGGC', 'GGGT'])
yyy = dnds(raw_seq, new_seq)

268.6666666666668


In [648]:
effect1[effect1['POS']==571815]

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT
2140,p3_D_S2,3,571815,0.0,674,668,6,0.898204,CCAGATGTAA,TCTGTTGTTT,start_lost,HIGH,cas9B,p.LeuAsp1?


In [656]:
for k, v in ttt.items():
    print(k, v[-1])

p3IPTG_A_S1 2.404754044239023
p3_D_S2 1.7427616926503313
p15_F_S1 2.3935483870967738
p15_F_S5 3.4130506790327924
p3IPTG_A_S1b 2.6608581600149894
p3_D_S3 2.117820324005891
p2_2_S3 2.3640178337267255
p2_2_S1 2.127371273712737
p2_2_S2 1.7000000000000002


In [56]:
df = pd.read_csv(, header=60, sep='\t')
df2 = df[(df['INFO'].str.contains('CIGAR=1X;'))].sample(n=150, random_state=1).copy()

SyntaxError: invalid syntax (<ipython-input-56-9a8f373c509c>, line 1)

In [57]:
pn, ps, pns, dn, ds, dns = 0, 0, 0, 0, 0, 0
for item in sorted(output[chromosome]):
    raw_seq, new_seq = get_new_sequence(args.reference, chromosome, output[chromosome][item][0].qualifiers[
        'locus_tag'][0], output[chromosome][item][1])
    newpn, newps, newdn, newds = dnds(raw_seq, new_seq)
    pn += newpn
    ps += newps
    dn += newdn
    ds += newds
if pn == 0 and ps == 0:
    pns, dns = '/', '/'
elif ps == 0:
    pns, dns = '+', '+'
elif pn == 0:
    pns, dns = '-', '-'
else:
    pns, dns = pn / ps, dn / ds
print args.query, chromosome, pn, ps, pns, dn, ds, dns

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(args.query, chromosome, pn, ps, pns, dn, ds, dns)? (<ipython-input-57-012244273d6f>, line 18)

In [58]:
for records in SeqIO.parse(, "genbank"):
    print(/)
    raw_seq, new_seq = get_new_sequence('./data/C5_refSeq.gb', 'C5', output[chromosome][item][0].qualifiers[
        'locus_tag'][0], output[chromosome][item][1])

SyntaxError: invalid syntax (<ipython-input-58-9fd2afaa1715>, line 1)